# DREAM data reduction - Normalized by proton charge

In [ ]:
import scipp as sc

from ess import dream, powder
import ess.dream.data  # noqa: F401
from ess.powder.types import *

## Create and configure the workflow

In [ ]:
workflow = dream.DreamGeant4Workflow(run_norm=powder.RunNormalization.proton_charge)

### Files

In [ ]:
workflow[Filename[SampleRun]] = dream.data.simulated_diamond_sample()
workflow[Filename[VanadiumRun]] = dream.data.simulated_vanadium_sample()
workflow[Filename[BackgroundRun]] = dream.data.simulated_empty_can()
workflow[CalibrationFilename] = None

### Masks

In [ ]:
workflow[TofMask] = lambda x: (x < sc.scalar(0.0, unit="ns")) | (
        x > sc.scalar(86e6, unit="ns")
)
workflow[TwoThetaMask] = None
workflow[WavelengthMask] = None
workflow = powder.with_pixel_mask_filenames(workflow, [])

### Other parameters

In [ ]:
workflow[NeXusDetectorName] = "mantle"
workflow[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.drop
workflow[DspacingBins] = sc.linspace("dspacing", 0.0, 2.3434, 201, unit="angstrom")

## Use the workflow

In [ ]:
results = workflow.compute([IofTof, ReducedTofCIF])

## Plot and save

In [ ]:
results[IofTof].hist().plot()

We can now save the result to disk:

In [ ]:
cif_data = results[ReducedTofCIF]

cif_data.comment = """This file was generated with the DREAM data reduction user guide
in the documentation of ESSdiffraction.
See https://scipp.github.io/essdiffraction/
"""
cif_data.save('reduced.cif')